In [1]:
!pip install eli5==0.13.0

# 한글 폰트 사용을 위한 라이브러리입니다.
!apt-get install -y fonts-nanum

Reading package lists... Done
Building dependency tree       
Reading state information... Done
fonts-nanum is already the newest version (20180306-3).
0 upgraded, 0 newly installed, 0 to remove and 14 not upgraded.


In [2]:
# visualization
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
fe = fm.FontEntry(
    fname=r'/usr/share/fonts/truetype/nanum/NanumGothic.ttf', # ttf 파일이 저장되어 있는 경로
    name='NanumBarunGothic')                        # 이 폰트의 원하는 이름 설정
fm.fontManager.ttflist.insert(0, fe)              # Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 10, 'font.family': 'NanumBarunGothic'}) # 폰트 설정
plt.rc('font', family='NanumBarunGothic')
import seaborn as sns

# utils
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import warnings;warnings.filterwarnings('ignore')
import re
import math
from geopy.geocoders import Nominatim
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor

# Model
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMRegressor
from sklearn.model_selection import KFold
from functools import partial
import eli5
from eli5.sklearn import PermutationImportance
import optuna

In [3]:
geo_local = Nominatim(user_agent='South Korea')

# 주소 -> 경도, 위도 반환하는 함수
def get_coordinates(addr):
    try:
        geo = geo_local.geocode(addr)
        x, y = geo.longitude, geo.latitude
        return x, y
    except:
        return np.nan, np.nan

# 주어진 행과 위치 데이터의 모든 지점 간의 거리 계산 및 조건을 만족하는 지점의 개수 반환
def calculate_distances_vectorized(row, df_loc, distance, alpha=0):
    row_coords = np.radians(np.array([row[0], row[1]]))
    df_coords = np.radians(df_loc[['좌표Y', '좌표X']].values.T)
    distances = np.linalg.norm(row_coords[:, np.newaxis] - df_coords, axis=0) * 6371000  # row_coords 브로드캐스팅
    return np.sum((distances + alpha <= distance).astype(int))

# 병렬 처리
def get_number_of_object(df_main, df_object, distances, alpha=0):
    df_main_loc = df_main[['좌표Y', '좌표X']]
    df_object_loc = df_object[['좌표Y', '좌표X']]

    with ThreadPoolExecutor() as executor:
        result = list(executor.map(lambda row_train: calculate_distances_vectorized(row_train, df_object_loc, distances),
                                    tqdm(df_main_loc.itertuples(index=False),
                                        total=len(df_main_loc), desc='Building Iteration', position=1)))

    df = pd.DataFrame(result)
    return df

# 위도, 경도로 두 지점간의 거리 계산
def haversine_distance(lat1, lon1, lat2, lon2):
    radius = 6371.0

    lat1 = math.radians(lat1)
    lon1 = math.radians(lon1)
    lat2 = math.radians(lat2)
    lon2 = math.radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = math.sin(dlat / 2) ** 2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    distance = radius * c
    return distance

In [4]:
# 필요한 데이터를 load 하겠습니다. 경로는 환경에 맞게 지정해주면 됩니다.
train_path = 'merge9.csv'
df_train = pd.read_csv(train_path)

# Train data와 Test data shape은 아래와 같습니다.
print('Train data shape : ', df_train.shape)

Train data shape :  (1128094, 40)


In [5]:
# 이상치 제거 방법에는 IQR을 이용하겠습니다.
def remove_outliers_iqr(dt, column_name):
    df = dt.query('is_test == 0')       # train data 내에 있는 이상치만 제거하도록 하겠습니다.
    df_test = dt.query('is_test == 1')

    Q1 = df[column_name].quantile(0.25)
    Q3 = df[column_name].quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    df = df[(df[column_name] >= lower_bound) & (df[column_name] <= upper_bound)]

    result = pd.concat([df, df_test])   # test data와 다시 합쳐주겠습니다.
    return result

In [6]:
# target 기준 이상치 제거
df_train = remove_outliers_iqr(df_train, 'target')

In [7]:
print(df_train.shape)

(1052472, 40)


In [8]:
# 데이터 수가 적은 높은 실거래가의 데이터를 복사하여 넣어줌
df_high_cases = df_train[df_train['target'] >= 1300000]
df_train = pd.concat([df_train, df_high_cases])
df_train = pd.concat([df_train, df_high_cases])

df_high_cases2 = df_train[df_train['target'] >= 1000000 & (df_train['target'] < 1300000)]
df_train = pd.concat([df_train, df_high_cases2])
df_train = pd.concat([df_train, df_high_cases2])

In [9]:
dt_train = df_train.query('is_test==0').reset_index()
dt_test = df_train.query('is_test==1').reset_index()

# 이제 is_test 칼럼은 drop해줍니다.
dt_train.drop(['is_test'], axis = 1, inplace=True)
dt_test.drop(['is_test'], axis = 1, inplace=True)
print(dt_train.shape, dt_test.shape)

(3129600, 40) (9272, 40)


In [10]:
final_cols = ['도로명_실거래가순위', '전용면적', 'k-복도유형', 'k-단지분류', '계약년',
                  '계약월', '동_실거래가순위', '좌표X', '좌표Y', '건축년도',
                  '부촌여부', '상위아파트여부', '대장아파트거리', '도로_실거래가순위', '구',
                  '주차대수', '인근지하철역개수', '브랜드명', '건물연식', '계약년월일',
                  'top아파트거리', 'target','재건축', '기준금리', 'CLI', '자치구별 지하철 승객 수', '공시지가', '아파트전세가격지수', '학군']

dt_train = dt_train[final_cols]
dt_test = dt_test[final_cols]

In [11]:
# 파생변수 제작으로 추가된 변수들이 존재하기에, 다시한번 연속형과 범주형 칼럼을 분리해주겠습니다.
numerical_columns_v2 = []
categorical_columns_v2 = []

for column in dt_train.columns:
    if pd.api.types.is_numeric_dtype(dt_train[column]):
        numerical_columns_v2.append(column)
    else:
        categorical_columns_v2.append(column)

print("수치형 변수:", numerical_columns_v2)
print("범주형 변수:", categorical_columns_v2)

수치형 변수: ['도로명_실거래가순위', '전용면적', '계약년', '계약월', '동_실거래가순위', '좌표X', '좌표Y', '건축년도', '부촌여부', '상위아파트여부', '대장아파트거리', '도로_실거래가순위', '주차대수', '인근지하철역개수', '건물연식', '계약년월일', 'top아파트거리', 'target', '재건축', '기준금리', 'CLI', '자치구별 지하철 승객 수', '공시지가', '아파트전세가격지수', '학군']
범주형 변수: ['k-복도유형', 'k-단지분류', '구', '브랜드명']


In [12]:
# 아래에서 범주형 변수들을 대상으로 레이블인코딩을 진행해 주겠습니다.
# 각 변수에 대한 LabelEncoder를 저장할 딕셔너리
label_encoders = {}

# Implement Label Encoding
for col in tqdm( categorical_columns_v2 ):
    lbl = LabelEncoder()

    # Label-Encoding을 fit
    lbl.fit( dt_train[col].astype(str) )
    dt_train[col] = lbl.transform(dt_train[col].astype(str))
    label_encoders[col] = lbl           # 나중에 후처리를 위해 레이블인코더를 저장해주겠습니다.

    # Test 데이터에만 존재하는 새로 출현한 데이터를 신규 클래스로 추가해줍니다.
    for label in np.unique(dt_test[col]):
      if label not in lbl.classes_: # unseen label 데이터인 경우
        lbl.classes_ = np.append(lbl.classes_, label)

    dt_test[col] = lbl.transform(dt_test[col].astype(str))

  0%|          | 0/4 [00:00<?, ?it/s]


TypeError: '<' not supported between instances of 'str' and 'float'

In [ ]:
assert dt_train.shape[1] == dt_test.shape[1]          # train/test dataset의 shape이 같은지 확인해주겠습니다.

In [ ]:
def train_valid_split(data_x, data_y, train_idx, valid_idx):
    x_train = data_x.iloc[train_idx]
    y_train = data_y[train_idx]
    x_valid = data_x.iloc[valid_idx]
    y_valid = data_y[valid_idx]
    return x_train, y_train, x_valid, y_valid

# k-fold cross-validation을 통한 모델 평가 함수
def evaluate(data_x, data_y, model, random_state=42, n_splits=5, test_x=None):
    kf = KFold(n_splits=n_splits, random_state=random_state, shuffle=True)

    oof_y = np.zeros(len(data_x))
    feature_importances = np.zeros((len(data_x.columns), n_splits))

    if test_x is not None:
        test_y = np.zeros((len(test_x), n_splits))

    for i, (train_index, valid_index) in enumerate(kf.split(data_x, data_y)):
        train_x, train_y, valid_x, valid_y = train_valid_split(data_x, data_y, train_index, valid_index)
        model.fit(train_x, train_y)

        oof_y[valid_index] = model.predict(valid_x) # out-of-fold 예측값
        feature_importances[:, i] = model.feature_importances_

        if test_x is not None:
            test_y[:, i] = model.predict(test_x)

        print(f'{i}th-fold Validation Score : ', mean_squared_error(valid_y, oof_y[valid_index], squared=False))

    # out-of-fold 예측값에 대한 RMSE score 계산
    score = mean_squared_error(data_y, oof_y, squared=False)
    print('OOF RMSE Score : ', score)

    return (oof_y, np.mean(test_y, axis=1), np.mean(feature_importances, axis=1)) if test_x is not None else (oof_y, np.mean(feature_importances, axis=1))

In [ ]:
# Target과 독립변수들을 분리해줍니다.
X_train_all = dt_train.drop(['target', '계약년월일'], axis=1)
y_train_all = dt_train['target']

# Custom validation split - 최근 20% 데이터를 validation set으로 나눔
dt_train = dt_train.sort_values('계약년월일')
cut = int(len(dt_train)*0.8)
train_split = dt_train[:cut]
valid_split = dt_train[cut:]

X_train = train_split.drop(['target', '계약년월일'], axis=1)
y_train = train_split['target']
X_val = valid_split.drop(['target', '계약년월일'], axis=1)
y_val = valid_split['target']

# LGBM 파라미터 설정
lgb_params = {
            'n_estimators': 2048,
            'force_col_wise': True
            }

model = LGBMRegressor(**lgb_params)

In [ ]:
# 모델 훈련 및 평가 후 oof 예측값 가져오기
# oof_pred, feature_importances = evaluate(X_train.copy(), y_train.copy(), model)

# 모델 학습 및 평가
model.fit(X_train, y_train)
pred = model.predict(X_val)

# 회귀 관련 metric을 통해 train/valid의 모델 적합 결과를 관찰합니다.
print(f'Validation RMSE: {mean_squared_error(y_val, pred, squared=False)}')

In [ ]:
# 위 feature importance를 시각화해봅니다.
importances = pd.Series(model.feature_importances_, index=list(X_train.columns))
importances = importances.sort_values(ascending=False)

plt.figure(figsize=(10,8))
plt.title("Feature Importances")
sns.barplot(x=importances, y=importances.index)
plt.show()

In [ ]:
# 전체 train 데이터로 학습된 모델을 저장
model.fit(X_train_all, y_train_all)
with open('saved_model.pkl', 'wb') as f:
    pickle.dump(model, f)

In [ ]:
# Validation dataset에 target과 pred 값을 채워주도록 하겠습니다.
X_val['target'] = y_val
X_val['pred'] = pred

# Squared_error를 계산하는 함수를 정의하겠습니다.
def calculate_se(target, pred):
    squared_errors = (target - pred) ** 2
    return squared_errors

# SE 계산
squared_errors = calculate_se(X_val['target'], X_val['pred'])
X_val['error'] = squared_errors

# Error가 큰 순서대로 sorting 해 보겠습니다.
X_val_sort = X_val.sort_values(by='error', ascending=False)
X_val_sort.head(10)

In [ ]:
X_val_sort_top100 = X_val.sort_values(by='error', ascending=False).head(100)        # 예측을 잘 하지못한 top 100개의 data
X_val_sort_tail100 = X_val.sort_values(by='error', ascending=False).tail(100)       # 예측을 잘한 top 100개의 data

# 해석을 위해 레이블인코딩 된 변수를 복원해줍니다.
error_top100 = X_val_sort_top100.copy()
for column in categorical_columns_v2 :     # 앞서 레이블 인코딩에서 정의했던 categorical_columns_v2 범주형 변수 리스트를 사용합니다.
    error_top100[column] = label_encoders[column].inverse_transform(X_val_sort_top100[column])

best_top100 = X_val_sort_tail100.copy()
for column in categorical_columns_v2 :     # 앞서 레이블 인코딩에서 정의했던 categorical_columns_v2 범주형 변수 리스트를 사용합니다.
    best_top100[column] = label_encoders[column].inverse_transform(X_val_sort_tail100[column])

display(error_top100.head(1))
display(best_top100.head(1))

In [ ]:
# 저장된 모델을 불러옵니다.
with open('saved_model.pkl', 'rb') as f:
    model = pickle.load(f)

In [ ]:
%%time
X_test = dt_test.drop(['target', '계약년월일'], axis=1)

# Test dataset에 대한 inference를 진행합니다.
real_test_pred = model.predict(X_test)

In [ ]:
# 앞서 예측한 예측값들을 저장합니다.
preds_df = pd.DataFrame(real_test_pred.astype(int), columns=["target"])
preds_df.to_csv('output2.csv', index=False)